## Resume - JobDescription Matching

## Introduction

Doc2Vec is an **unsupervised learning algorithm** that allows **documents** (instead of just words) to be represented as numerical vectors in a continuous vector space. It is an extension of **Word2Vec**, but instead of learning embeddings for individual words, it learns fixed-length representations for entire documents.

It was introduced in the paper:
**"Distributed Representations of Sentences and Documents"** by *Quoc V. Le and Tomas Mikolov* (Google), published in 2014.


## **How Does Doc2Vec Work?**
Doc2Vec builds on Word2Vec by introducing a **document vector** that captures the semantics of an entire document. There are two main models, we chose:

1. **PV-DM (Paragraph Vector - Distributed Memory)**
   - Works similarly to **CBOW (Continuous Bag of Words)** in Word2Vec.
   - Predicts a missing word in a sentence using both **word embeddings** and a **document embedding**.
   - The document vector acts as a **memory** of the entire document, making the model capture context better.

## Step-to-Step

To implement CV Job Matching using Doc2Vec, we start by importing the necessary libraries and loading the job data from a CSV file. We preprocess the data, keeping only the relevant columns, and merge them into a new column called 'data.' Then, we tokenize the words in the 'data' column and tag them with unique identifiers using the TaggedDocument class.

Next, we initialize the Doc2Vec model with specific parameters, such as the vector size, minimum count, and number of epochs. We build the vocabulary by feeding the tagged data to the model, and then train the model on the tagged data.

After training, we save the model for future use. To match a resume with a job description, we load the saved model and preprocess the resume and job description text. We convert them to lowercase, remove punctuation and numerical values.

Using the trained model, we infer the document vectors for the resume and job description. Then, we calculate *cosine similarity*between the two vectors to determine the match between the resume and the job description. The cosine similarity score ranges from -1 to 1, with 1 indicating a perfect match and -1 indicating no similarity.

By employing Doc2Vec and cosine similarity, this approach enables efficient and effective matching between job descriptions and resumes, helping to streamline the job application process and enhance the chances of finding the right candidates for specific positions.

Finally, the author also employs Gauge chart from Plotly to show the matching percentage with threshold that users could consider modifying thier CV to pass Application Tracking System (TSA) from the majority of employers.

## Coding
### 1. Set up

In [1]:
# Install all dependencies

!pip install gensim
!pip install nltk
!pip install pandas
!pip install numpy
!pip install requests
!pip install PyPDF2
!pip install termcolor
!pip install plotly
!pip install spacy
!pip install nbformat



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Import libraries
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
#from nltk.tokenize import word_tokenize
from numpy.linalg import norm
from termcolor import colored
import pandas as pd
import numpy as np
import PyPDF2
import re
import plotly.graph_objects as go
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### 2. Prepare data
This dataset that we trained our model contains current job postings available.You can follow this link to download: 
[New York Job Posting Dataset](https://data.world/city-of-ny/kpav-sd4t)

In [3]:
# Load data
df = pd.read_csv('./dataset/nyc-jobs-1.csv')
# Check data
df.head()

,Job ID,Agency,Posting Type,# Of Positions,Business Title,Civil Service Title,Title Code No,Level,Job Category,Full-Time/Part-Time indicator,...,Additional Information,To Apply,Hours/Shift,Work Location 1,Recruitment Contact,Residency Requirement,Posting Date,Post Until,Posting Updated,Process Date
0,87990,DEPARTMENT OF BUSINESS SERV.,Internal,1,Account Manager,CONTRACT REVIEWER (OFFICE OF L,40563,1,,,...,"Salary range for this position is: $42,405 - $...",,,,NaN,New York City residency is generally required ...,2011-06-24T00:00:00,NaN,2011-06-24T00:00:00,2018-07-17T00:00:00
1,97899,DEPARTMENT OF BUSINESS SERV.,Internal,1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",ADMINISTRATIVE BUSINESS PROMOT,10009,M3,,F,...,,"In addition to applying through this website, ...",,,NaN,New York City residency is generally required ...,2012-01-26T00:00:00,NaN,2012-01-26T00:00:00,2018-07-17T00:00:00
2,102221,DEPT OF ENVIRONMENT PROTECTION,External,1,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,20616,0,,F,...,Appointments are subject to OMB approval,click the apply now button,35 hours per week/day,,NaN,New York City Residency is not required for th...,2012-06-21T00:00:00,NaN,2012-09-07T00:00:00,2018-07-17T00:00:00
3,102221,DEPT OF ENVIRONMENT PROTECTION,Internal,1,Project Specialist,ENVIRONMENTAL ENGINEERING INTE,20616,0,,F,...,Appointments are subject to OMB approval,click the apply now button,35 hours per week/day,,NaN,New York City Residency is not required for th...,2012-06-21T00:00:00,NaN,2012-09-07T00:00:00,2018-07-17T00:00:00
4,114352,DEPT OF ENVIRONMENT PROTECTION,Internal,5,Deputy Plant Chief,SENIOR STATIONARY ENGINEER (EL,91639,0,,F,...,Appointments are subject to OMB approval Fo...,"Click ""Apply Now"" button",40 per week / day,Various,NaN,New York City residency is generally required ...,2012-12-12T00:00:00,NaN,2012-12-13T00:00:00,2018-07-17T00:00:00


In [4]:
# Show column name
df.columns

Index(['Job ID', 'Agency', 'Posting Type', '# Of Positions', 'Business Title',
       'Civil Service Title', 'Title Code No', 'Level', 'Job Category',
       'Full-Time/Part-Time indicator', 'Salary Range From', 'Salary Range To',
       'Salary Frequency', 'Work Location', 'Division/Work Unit',
       'Job Description', 'Minimum Qual Requirements', 'Preferred Skills',
       'Additional Information', 'To Apply', 'Hours/Shift', 'Work Location 1',
       'Recruitment Contact', 'Residency Requirement', 'Posting Date',
       'Post Until', 'Posting Updated', 'Process Date'],
      dtype='object')

Keep only some columns to train the model

In [5]:
df =df[['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills']]
df.head()

,Business Title,Job Description,Minimum Qual Requirements,Preferred Skills
0,Account Manager,Division of Economic & Financial Opportunity (...,1.\tA baccalaureate degree from an accredited ...,â€¢\tExcellent interpersonal and organizationa...
1,"EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT",The New York City Department of Small Business...,1. A baccalaureate degree from an accredited c...,
2,Project Specialist,"Under direct supervision, perform elementary e...",A Baccalaureate degree from an accredited coll...,
3,Project Specialist,"Under direct supervision, perform elementary e...",A Baccalaureate degree from an accredited coll...,
4,Deputy Plant Chief,"Under general direction, is in responsible cha...",1. Six years of full-time satisfactory experie...,


In [6]:
# Create a new column called 'data' and merge the values of the other columns into it
df['data'] = df[['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills']].apply(lambda x: ' '.join(x.dropna().astype(str)), axis=1)
# Drop the individual columns if you no longer need them
df.drop(['Business Title', 'Job Description', 'Minimum Qual Requirements', 'Preferred Skills'], axis=1, inplace=True)
# Preview the updated dataframe
print(df.head())

                                                data
0  Account Manager Division of Economic & Financi...
1  EXECUTIVE DIRECTOR, BUSINESS DEVELOPMENT The N...
2  Project Specialist Under direct supervision, p...
3  Project Specialist Under direct supervision, p...
4  Deputy Plant Chief Under general direction, is...


#### 3. Tokenization model initialization and vocab building
We nitialize the Doc2Vec model with specific parameters.


**Parameters** of Doc2Vec are as follows: 

- `vector_size`: Dimensionality of the feature vectors. Default: 100.
- `window`: The window refers to the maximum distance between the target word and its context words within a sentence. Default: 5.
- `min_count`: Ignores all words with a total frequency lower than this. Default: 5.
- `epochs`: Number of iterations (epochs) over the corpus. Defaults to 5 for PV-DBOW and 10 for PV-DM.
- `dm`: Defines the training algorithm. If `dm = 1`, the Distributed Memory (PV-DM) model is used. If `dm = 0`, the Distributed Bag of Words (PV-DBOW) model is used. Default: 1 (PV-DM).
- `dbow_words`: If set to 1, trains word vectors (in addition to document vectors) using the PV-DBOW algorithm. Default: 0 (False).
- `dm_mean`: If set to 1, uses the mean of the context word vectors instead of concatenation when inferring vectors in the PV-DM model. Default: 0 (False).
- `dm_concat`: If set to 1, concatenates the document and context word vectors when inferring vectors in the PV-DM model. Default: 0 (False).
- `dm_tag_count`: Expected number of document tags per document, when using the PV-DM algorithm. Default: 1.
- `dbow_tag_count`: Expected number of document tags per document, when using the PV-DBOW algorithm. Default: 1.
- `alpha`: The initial learning rate. Default: 0.025.
- `min_alpha`: The learning rate will linearly drop to `min_alpha` as training progresses. Default: 0.0001.
- `hs`: If set to 1, hierarchical softmax activation function will be used. Default: 0 (Negative Sampling).
- `negative`: If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drawn. Default: 5.
- `ns_exponent`: The exponent used to shape the negative sampling distribution. Default: 0.75.


I deleted the codes to not confuse

#### 4 Train and save the model
Train the model on tagged data.

In [7]:
# # Train the model
# for epoch in range(model.epochs):
#     print(f"Training epoch {epoch+1}/{model.epochs}")
#     model.train(tagged_data, 
#                 total_examples=model.corpus_count, 
#                 epochs=model.epochs)

# model.save('cv_job_maching.model')
# print("Model saved")

#### 6. Inputs of CV and JD

- **Resume**

I  upload my CV PDF file, so I use PyPDF2 to extract data. You can also change how to read inputs appropreately.

In [8]:
pdf = PyPDF2.PdfReader('./CV/Yakhou_Yousra___QRT.pdf')
resume = ""
for i in range(len(pdf.pages)):
    pageObj = pdf.pages[i]
    resume += pageObj.extract_text()

In [9]:
resume

'Yousra Yakhou\nArtificial Intelligence Student\nćyakhouyousra@gmail.com Ħ+33668573758 ]www.linkedin.com/in/yousrayakhou agithub.com/yousrayk\nEDUCATION\nEcole CentraleSupelec September 2024 – December 2025\nMaster of Engineering M2 in Artificial Intelligence Gif-sur-Yvette, France\n•Ranked among top 4% for admission to the Master’s.\nESSEC Business School 2023 – 2024\nBBA AFRIQUE Rabat, Morocco\n•Ranked among top 10% for admission to the ESSEC Exchange Program (International Negotiation and M&A).\nEcole Centrale Casablanca September 2022 – June 2024\nBachelor in Computer Science Casablanca, Morocco\n•Awarded scholarship for best exam results in class of 180+ students.\nEXPERIENCE\nBCG X - Paris, France October 2024 – Ongoing\nProfessional Development Program\n•Engaged in case studies program with a group of women mentored by BCG consultants, focusing on Data Science.\nOracle - Casablanca, Morocco June 2024 – September 2024\nResearch Assistant in Machine Learning\n•Developed an AI assi

- **Job Description**:

From my perspective, I believe THE RECRUITER will copy and paste the JD into textbox to check the matching percentage, so I will have JD Input in text as below.

In [10]:
# JD by input text:
jd = input("Paste your JD here: ")

In [11]:
jd = """
Proactively engage with clients in the branch and deliver an outstanding service experience through completing transactions and by identifying opportunities for advice, solutions, digital enablement and partner introductions
Focus on education and demonstration, leverage technology to deliver a memorable client experience, drive solutions and retain business
Contribute to team results by listening and spotting opportunities to offer additional advice, introduce clients to the capability of RBC partners, or personally fulfil client solutions
Proactively take ownership of resolving and preventing client banking problems
Cultivate and maintain relationships with partners to work as one RBC team
Manage risks by adhering to compliance routines, processes, and controls to protect client and shareholder interests while completing transactions
What do you need to succeed?

Must-have

Goal-oriented individual with a demonstrated passion for putting clients first.
Drive and self-motivation, as well as excellent communication skills and emotional intelligence
Digital literacy across a broad range of devices (i.e., smartphones, tablets, laptops, etc.)
Personal flexibility to work flex hours
Eagerness to learn and determination to succeed
Confidence and ability to learn financial concepts and willingness to obtain the Investment Funds in Canada or the Canadian Securities Course
Nice-to-have

Track record in building rapport and maintaining client relationships within the financial, service or retail industry
Mutual Funds accreditation
Is this job right for you? Check out our video and decide for yourself!

What's in it for you?

We thrive on the challenge to be our best, progressive thinking to keep growing, and working together to deliver trusted advice to help our clients thrive and communities prosper. We care about each other, reaching our potential, making a difference to our communities, and achieving success that is mutual.

A comprehensive Total Rewards Program including bonuses and flexible benefits, competitive compensation, commissions, and stock where applicable
A world-class training program in financial services
Excellent career development and access to a variety of job opportunities across business and geographies
Leaders who support your development through coaching and managing opportunities
Work in a dynamic, collaborative, progressive, and high-performing team
We also strive to provide an accessible candidate experience for our prospective employees with different abilities. Please let us know if you need any accommodations during the recruitment process.

Join our Talent Community

Stay in-the-know about great career opportunities at RBC. Sign up and get customized info on our latest jobs, career tips and Recruitment events that matter to you.

Expand your limits and create a new future together at RBC. Find out how we use our passion and drive to enhance the well-being of our clients and communities at rbc.com/careers.
"""

- **Develop a function to pre-process input text**:

In [12]:
def preprocess_text(text):
    # Convert the text to lowercase
    text = text.lower()
    
    # Remove punctuation from the text
    text = re.sub('[^a-z]', ' ', text)
    
    # Remove numerical values from the text
    text = re.sub(r'\d+', '', text)
    
    # Remove extra whitespaces
    text = ' '.join(text.split())
    
    return text

In [13]:
# Apply to CV and JD
input_CV = preprocess_text(resume)
input_JD = preprocess_text(jd)

#### 7. Matching
Using the trained model, we infer the document vectors for the resume and job description. Then, we calculate the cosine similarity between the two vectors to determine the match between the resume and the job description.

In [14]:
# Model evaluation
model = Doc2Vec.load('cv_job_maching.model')
v1 = model.infer_vector(input_CV.split())
v2 = model.infer_vector(input_JD.split())
similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
print(round(similarity, 2))

24.76


#### 8. Visualization and Notification

In [15]:
import plotly.graph_objects as go

# Visualization from recruiter's perspective
fig = go.Figure(go.Indicator(
    domain={'x': [0, 1], 'y': [0, 1]},
    value=similarity,
    mode="gauge+number",
    title={'text': "Candidate Matching Score (%)"},
    gauge={
        'axis': {'range': [0, 100]},
        'steps': [
            {'range': [0, 50], 'color': "#FF6347"},  # Red for low match (reject)
            {'range': [50, 70], 'color': "#FFD700"},  # Yellow for moderate match (review carefully)
            {'range': [70, 100], 'color': "#32CD32"}  # Green for strong match (proceed)
        ],
        'threshold': {'line': {'color': "blue", 'width': 4}, 'thickness': 0.75, 'value': similarity}
    }
))

fig.update_layout(
    width=600, 
    height=400,
    title={
        'text': "Recruiter Decision Gauge",
        'x': 0.5,  # Center title
        'xanchor': 'center'
    }
)

fig.show()

# Print notification for recruiter
if similarity < 50:
    print(colored("Low match, consider rejecting this candidate.", "red", attrs=["bold"]))
elif similarity >= 50 and similarity < 70:
    print(colored("Moderate match, review carefully before deciding.", "yellow", attrs=["bold"]))
else:
    print(colored("Strong match! Consider proceeding with this candidate.", "green", attrs=["bold"]))


Low match, consider rejecting this candidate.


In [ ]:

def match_resume_to_jd(resume_text, jd_text):
    def preprocess_text(text):
        text = text.lower()
        text = re.sub('[^a-z]', ' ', text)
        text = re.sub(r'\d+', '', text)
        text = ' '.join(text.split())
        return text

    model = Doc2Vec.load('cv_job_maching.model')
    v1 = model.infer_vector(preprocess_text(resume_text).split())
    v2 = model.infer_vector(preprocess_text(jd_text).split())
    similarity = 100*(np.dot(np.array(v1), np.array(v2))) / (norm(np.array(v1)) * norm(np.array(v2)))
    return round(similarity, 2)



match_resume_to_jd(resume, jd)
